In [ ]:
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from torch.utils.data import DataLoader
from poc_data import PocDataset, display_sample


NUM_CASES = 2**2

poc_raw_dataset = PocDataset(num_cases=NUM_CASES, test_set=True, repays=False)
data_loader = DataLoader(dataset=poc_raw_dataset,
                        batch_size=NUM_CASES,
                        shuffle=False,
                        pin_memory=True)

all_data, all_data_length, all_event, all_tte, all_meta = next(iter(data_loader))

In [ ]:
print(all_data.shape)
print(all_data_length.shape)
print(all_event.shape)

In [ ]:
from losses import loss_1_batch, loss_2_batch, loss_3_batch

input_size = 5
output_size = input_size
MAX_LENGTH = 36
NUM_CAUSES = 3
hidden_size_encoder = 512
hidden_size_attention = 512
fc_size_encoder = 512
SIGMA = 1

LOSS_1_AMPLIFIER = 1
LOSS_2_AMPLIFIER = 1
LOSS_3_AMPLIFIER = 0.1

DEVICE = 'cpu'

In [ ]:
## Sanity check on only events for loss 1
BATCH_LENGTH = 1

first_hitting_time_batch = torch.zeros(BATCH_LENGTH, MAX_LENGTH*NUM_CAUSES)

first_hitting_time_batch[0,16] = 0
first_hitting_time_batch[0,15] = 0
first_hitting_time_batch[0,14] = 1
first_hitting_time_batch[0,13] = 0
first_hitting_time_batch[0,12] = 0

batch_event = torch.zeros(BATCH_LENGTH, 1)
batch_event[0] = 0

batch_tte = torch.zeros(BATCH_LENGTH, 1)
batch_tte[0] = 15

loss1 = LOSS_1_AMPLIFIER*loss_1_batch(first_hitting_time_batch, batch_event, batch_tte, MAX_LENGTH)
print("loss1=", loss1)

In [ ]:
## Check including censoring events
BATCH_LENGTH = 1

first_hitting_time_batch = torch.zeros(BATCH_LENGTH, MAX_LENGTH*NUM_CAUSES)

first_hitting_time_batch[0,16] = 0
first_hitting_time_batch[0,15] = 0
first_hitting_time_batch[0,14] = 0
first_hitting_time_batch[0,13] = 0.02
first_hitting_time_batch[0,12] = 0.23
first_hitting_time_batch[0,9] = 0.25
first_hitting_time_batch[0,5] = 0.25

batch_event = torch.zeros(BATCH_LENGTH, 1)
batch_event[0] = 3

batch_tte = torch.zeros(BATCH_LENGTH, 1)
batch_tte[0] = 15

loss1 = LOSS_1_AMPLIFIER*loss_1_batch(first_hitting_time_batch, batch_event, batch_tte, MAX_LENGTH)
print("loss1=", loss1)

In [ ]:
## Checking full repays
BATCH_LENGTH = 1

first_hitting_time_batch = torch.zeros(BATCH_LENGTH, MAX_LENGTH*NUM_CAUSES)

first_hitting_time_batch[0,35 + 2*MAX_LENGTH] = 1
first_hitting_time_batch[0,34 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,15 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,14 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,13 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,12 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,9 + 2*MAX_LENGTH] = 0
first_hitting_time_batch[0,5 + 2*MAX_LENGTH] = 0

batch_event = torch.zeros(BATCH_LENGTH, 1)
batch_event[0] = 2

batch_tte = torch.zeros(BATCH_LENGTH, 1)
batch_tte[0] = 36

loss1 = LOSS_1_AMPLIFIER*loss_1_batch(first_hitting_time_batch, batch_event, batch_tte, MAX_LENGTH)
print("loss1=", loss1)

In [ ]:
from utils import plot_fht, plot_cif, plot_fht_and_cif

first_hitting_time = first_hitting_time_batch[0]
plot_fht_and_cif(first_hitting_time, MAX_LENGTH)

#We bedoelen dat als bij event 0, de index 14 op 1 staat, dan is het na de 15de observation dat het gaat falen, dus na het data-inputje dat op index 14 staat, maar dus de 15de input is

In [ ]:
loss1 = LOSS_1_AMPLIFIER*loss_1_batch(first_hitting_time_batch, batch_event, batch_tte, MAX_LENGTH)
print("loss1=", loss1)

In [ ]:
loss2 = LOSS_2_AMPLIFIER*loss_2_batch(first_hitting_time_batch, batch_event, batch_tte, NUM_CAUSES, MAX_LENGTH, SIGMA, DEVICE)
print("loss2=", loss2)

In [ ]:
#loss3 = LOSS_3_AMPLIFIER*loss_3_batch(output_batch, batch_data.detach())
#print("loss3=", loss3)